In [1]:
import os
os.chdir("../")

import semiolog as slg

In [2]:
semiotic = slg.Cenematic("en_bnc",requested_cpu=4)

Vocabulary will not be loaded from file.



In [ ]:
semiotic.vocab.build(vocab_size = 1000, parallel=True, corpus_length=80000, save=True, save_step=100)